# Importing Libraries

In [142]:
import pandas as pd
import numpy as np
import folium
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [114]:
df = pd.read_csv("postal_codes_with_lat_lng.csv")
df.rename(columns={'Neighborhood':"Neighbourhood"},inplace=True)
df.head()

PostalCode            Borough                       Neighbourhood  \
0        M3A         North York                           Parkwoods   
1        M4A         North York                    Victoria Village   
2        M5A   Downtown Toronto          Harbourfront,  Regent Park   
3        M6A         North York   Lawrence Heights,  Lawrence Manor   
4        M7A       Queen's Park                        Queen's Park   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

In [115]:
toronto_df = df[df['Borough'].str.contains('Toronto')]
toronto_df.reset_index(inplace=True)
toronto_df.shape

(38, 6)

In [116]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


# Create a map of Toronto with neighborhoods superimposed on top.

In [117]:
map_toronto = folium.Map(location=[latitude_toronto,longitude_toronto],zoom_start=11)
# add markers
for lat,lng,borough,neighborhood in zip(toronto_df['Latitude'],toronto_df['Longitude'],toronto_df['Borough'],toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
     [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)
map_toronto
    

## Define Foursquare Credentials and Version

In [118]:
CLIENT_ID = '1QGUT0CAWRYWHGYENM4BRUXFHSG00HRFDJ2QDLIV3EPQDRW5'
CLIENT_SECRET = 'LSF44YVP5FAD0HPHI4HHP2YPMWVOY5O2F11UP5AR0YA1GRWO'
VERSION = '20190310'
LIMIT = 50


##  Explore Neighborhoods in Toronto

In [119]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )


 Harbourfront,  Regent Park
 Ryerson,  Garden District
 St. James Town
 The Beaches
 Berczy Park
 Central Bay Street
 Christie
 Adelaide,  King,  Richmond
 Dovercourt Village,  Dufferin
 Harbourfront East,  Toronto Islands,  Union Station
 Little Portugal,  Trinity
 The Danforth West,  Riverdale
 Design Exchange,  Toronto Dominion Centre
 Brockton,  Exhibition Place,  Parkdale Village
 The Beaches West,  India Bazaar
 Commerce Court,  Victoria Hotel
 Studio District
 Lawrence Park
 Roselawn
 Davisville North
 Forest Hill North,  Forest Hill West
 High Park,  The Junction South
 North Toronto West
 The Annex,  North Midtown,  Yorkville
 Parkdale,  Roncesvalles
 Davisville
 Harbord,  University of Toronto
 Runnymede,  Swansea
 Moore Park,  Summerhill East
 Chinatown,  Grange Park,  Kensington Market
 Deer Park,  Forest Hill SE,  Rathnelly,  South Hill,  Summerhill West
 CN Tower,  Bathurst Quay,  Island airport,  Harbourfront West,  King and Spadina,  Railway Lands,  South Niagara
 Rosed

In [122]:
print(toronto_venues.shape)
toronto_venues.head()

(1163, 7)


Neighbourhood  Neighbourhood Latitude  \
0   Harbourfront,  Regent Park                43.65426   
1   Harbourfront,  Regent Park                43.65426   
2   Harbourfront,  Regent Park                43.65426   
3   Harbourfront,  Regent Park                43.65426   
4   Harbourfront,  Regent Park                43.65426   

   Neighbourhood Longitude                                            Venue  \
0               -79.360636                                 Roselle Desserts   
1               -79.360636                                    Tandem Coffee   
2               -79.360636  Toronto Cooper Koo Family Cherry St YMCA Centre   
3               -79.360636                              Body Blitz Spa East   
4               -79.360636                               Morning Glory Cafe   

   Venue Latitude  Venue Longitude        Venue Category  
0       43.653447       -79.362017                Bakery  
1       43.653559       -79.361809           Coffee Shop  
2       43.653191       -79.357947  Gym / Fitness Center  
3       43.654735       -79.359874                   Spa  
4       43.653947       -79.361149        Breakfast Spot

In [123]:
toronto_venues.groupby('Neighbourhood').count()

Neighbourhood Latitude  \
Neighbourhood                                                                
 Adelaide,  King,  Richmond                                             50   
 Berczy Park                                                            50   
 Brockton,  Exhibition Place,  Parkdale Village                         20   
 Business Reply Mail Processing Centre 969 Eastern                      17   
 CN Tower,  Bathurst Quay,  Island airport,  Ha...                      14   
 Cabbagetown,  St. James Town                                           46   
 Central Bay Street                                                     50   
 Chinatown,  Grange Park,  Kensington Market                            50   
 Christie                                                               16   
 Church and Wellesley                                                   50   
 Commerce Court,  Victoria Hotel                                        50   
 Davisville                                                             33   
 Davisville North                                                       10   
 Deer Park,  Forest Hill SE,  Rathnelly,  South...                      15   
 Design Exchange,  Toronto Dominion Centre                              50   
 Dovercourt Village,  Dufferin                                          18   
 First Canadian Place,  Underground city                                50   
 Forest Hill North,  Forest Hill West                                    4   
 Harbord,  University of Toronto                                        33   
 Harbourfront East,  Toronto Islands,  Union St...                      50   
 Harbourfront,  Regent Park                                             49   
 High Park,  The Junction South                                         25   
 Lawrence Park                                                           3   
 Little Portugal,  Trinity                                              50   
 Moore Park,  Summerhill East                                            1   
 North Toronto West                                                     17   
 Parkdale,  Roncesvalles                                                16   
 Rosedale                                                                4   
 Roselawn                                                                2   
 Runnymede,  Swansea                                                    38   
 Ryerson,  Garden District                                              50   
 St. James Town                                                         50   
 Stn A PO Boxes 25 The Esplanade                                        50   
 Studio District                                                        40   
 The Annex,  North Midtown,  Yorkville                                  24   
 The Beaches                                                             6   
 The Beaches West,  India Bazaar                                        19   
 The Danforth West,  Riverdale                                          43   

                                                    Neighbourhood Longitude  \
Neighbourhood                                                                 
 Adelaide,  King,  Richmond                                              50   
 Berczy Park                                                             50   
 Brockton,  Exhibition Place,  Parkdale Village                          20   
 Business Reply Mail Processing Centre 969 Eastern                       17   
 CN Tower,  Bathurst Quay,  Island airport,  Ha...                       14   
 Cabbagetown,  St. James Town                                            46   
 Central Bay Street                                                      50   
 Chinatown,  Grange Park,  Kensington Market                             50   
 Christie                                                                16   
 Church and Wellesley                                         

In [124]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 213 uniques categories.


## Analyze Each Neighborhood

In [125]:

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

Neighbourhood  Adult Boutique  Afghan Restaurant  Airport  \
0   Harbourfront,  Regent Park               0                  0        0   
1   Harbourfront,  Regent Park               0                  0        0   
2   Harbourfront,  Regent Park               0                  0        0   
3   Harbourfront,  Regent Park               0                  0        0   
4   Harbourfront,  Regent Park               0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant     ...       Toy / Game Store  \
0                 0                    0     ...                      0   
1                 0                    0     ...                      0   
2                 0                    0     ...                      0   
3                 0                    0     ...                      0   
4                 0                    0     ...                      0   

   Trail  Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0      0              0                              0                 0   
1      0              0                              0                 0   
2      0              0                              0                 0   
3      0              0                              0                 0   
4      0              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Wine Shop  Wings Joint  Yoga Studio  
0                      0         0          0            0            0  
1                      0         0          0            0            0  
2                      0         0          0            0            0  
3                      0         0          0            0            0  
4                      0         0          0            0            0  

[5 rows x 214 columns]

In [88]:
toronto_onehot.shape

(1163, 214)

In [126]:
# group by neighborhood by taking mean
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

Neighbourhood  Adult Boutique  \
0                          Adelaide,  King,  Richmond            0.00   
1                                         Berczy Park            0.00   
2      Brockton,  Exhibition Place,  Parkdale Village            0.00   
3    Business Reply Mail Processing Centre 969 Eas...            0.00   
4    CN Tower,  Bathurst Quay,  Island airport,  H...            0.00   
5                        Cabbagetown,  St. James Town            0.00   
6                                  Central Bay Street            0.00   
7         Chinatown,  Grange Park,  Kensington Market            0.00   
8                                            Christie            0.00   
9                                Church and Wellesley            0.02   
10                    Commerce Court,  Victoria Hotel            0.00   
11                                         Davisville            0.00   
12                                   Davisville North            0.00   
13   Deer Park,  Forest Hill SE,  Rathnelly,  Sout...            0.00   
14          Design Exchange,  Toronto Dominion Centre            0.00   
15                      Dovercourt Village,  Dufferin            0.00   
16            First Canadian Place,  Underground city            0.00   
17               Forest Hill North,  Forest Hill West            0.00   
18                    Harbord,  University of Toronto            0.00   
19   Harbourfront East,  Toronto Islands,  Union S...            0.00   
20                         Harbourfront,  Regent Park            0.00   
21                     High Park,  The Junction South            0.00   
22                                      Lawrence Park            0.00   
23                          Little Portugal,  Trinity            0.00   
24                       Moore Park,  Summerhill East            0.00   
25                                 North Toronto West            0.00   
26                            Parkdale,  Roncesvalles            0.00   
27                                           Rosedale            0.00   
28                                           Roselawn            0.00   
29                                Runnymede,  Swansea            0.00   
30                          Ryerson,  Garden District            0.00   
31                                     St. James Town            0.00   
32                    Stn A PO Boxes 25 The Esplanade            0.00   
33                                    Studio District            0.00   
34              The Annex,  North Midtown,  Yorkville            0.00   
35                                        The Beaches            0.00   
36                    The Beaches West,  India Bazaar            0.00   
37                      The Danforth West,  Riverdale            0.00   

    Afghan Restaurant   Airport  Airport Food Court  Airport Gate  \
0                0.00  0.000000            0.000000      0.000000   
1                0.00  0.000000            0.000000      0.000000   
2                0.00  0.000000            0.000000      0.000000   
3                0.00  0.000000            0.000000      0.000000   
4                0.00  0.071429            0.071429      0.071429   
5                0.00  0.000000            0.000000      0.000000   
6                0.00  0.000000            0.000000      0.000000   
7                0.00  0.000000            0.000000      0.000000   
8                0.00  0.000000            0.000000      0.000000   
9                0.02  0.000000            0.000000      0.000000   
10               0.00  0.000000            0.000000      0.000000   
11               0.00  0.000000            0.000000      0.000000   
12               0.00  0.000000            0.000000      0.000000   
13               0.00  0.000000            0.000000      0.000000   
14               0.00  0.000000            0.000000      0.000000   
15               0.00  0.000000            0.000000      0.000000   
16               0.

#### Let's print each neighborhood along with the top 5 most common venues

In [127]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide,  King,  Richmond----
                 venue  freq
0                 Café  0.06
1           Steakhouse  0.06
2           Restaurant  0.04
3  American Restaurant  0.04
4          Pizza Place  0.04


---- Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.06
2  Italian Restaurant  0.04
3      Farmers Market  0.04
4          Steakhouse  0.04


---- Brockton,  Exhibition Place,  Parkdale Village----
            venue  freq
0  Breakfast Spot  0.10
1     Coffee Shop  0.10
2            Café  0.10
3   Burrito Place  0.05
4         Stadium  0.05


---- Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1    Recording Studio  0.06
2              Garden  0.06
3       Garden Center  0.06
4          Skate Park  0.06


---- CN Tower,  Bathurst Quay,  Island airport,  Harbourfront West,  King and Spadina,  Railway Lands,  South Niagara----
              venue  freq
0  Airport Termina

In [129]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [130]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighbourhood 1st Most Common Venue  \
0                         Adelaide,  King,  Richmond                  Café   
1                                        Berczy Park           Coffee Shop   
2     Brockton,  Exhibition Place,  Parkdale Village           Coffee Shop   
3   Business Reply Mail Processing Centre 969 Eas...    Light Rail Station   
4   CN Tower,  Bathurst Quay,  Island airport,  H...        Airport Lounge   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0            Steakhouse           Pizza Place                 Hotel   
1          Cocktail Bar                  Café           Cheese Shop   
2                  Café        Breakfast Spot           Yoga Studio   
3                  Park      Recording Studio            Skate Park   
4      Airport Terminal       Airport Service       Harbor / Marina   

  5th Most Common Venue 6th Most Common Venue   7th Most Common Venue  \
0        Breakfast Spot            Restaurant        Asian Restaurant   
1            Steakhouse    Seafood Restaurant          Farmers Market   
2                   Bar                   Gym  Furniture / Home Store   
3                   Spa        Farmers Market    Fast Food Restaurant   
4      Sculpture Garden              Boutique           Boat or Ferry   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0   American Restaurant             Gastropub            Coffee Shop  
1            Restaurant    Italian Restaurant                 Bakery  
2    Italian Restaurant             Nightclub      Convenience Store  
3               Brewery         Burrito Place          Auto Workshop  
4          Airport Gate                 Plane                Airport

# Cluster Neighborhood

In [131]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 
# to change use .astype()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       4, 0, 1, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [135]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = toronto_df

 merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

index PostalCode            Borough                Neighbourhood  \
0      2        M5A   Downtown Toronto   Harbourfront,  Regent Park   
1      9        M5B   Downtown Toronto    Ryerson,  Garden District   
2     15        M5C   Downtown Toronto               St. James Town   
3     19        M4E       East Toronto                  The Beaches   
4     20        M5E   Downtown Toronto                  Berczy Park   

    Latitude  Longitude  Cluster_Labels 1st Most Common Venue  \
0  43.654260 -79.360636               0           Coffee Shop   
1  43.657162 -79.378937               0                  Café   
2  43.651494 -79.375418               0           Coffee Shop   
3  43.676357 -79.293031               0           Coffee Shop   
4  43.644771 -79.373306               0           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                Bakery                  Park                   Pub   
1           Coffee Shop        Clothing Store      Ramen Restaurant   
2            Restaurant                 Hotel             Gastropub   
3     Health Food Store                   Pub  Other Great Outdoors   
4          Cocktail Bar                  Café           Cheese Shop   

       5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                       Café        Breakfast Spot            Restaurant   
1  Middle Eastern Restaurant               Theater             Gastropub   
2        Japanese Restaurant        Farmers Market              Beer Bar   
3               Neighborhood           Yoga Studio         Deli / Bodega   
4                 Steakhouse    Seafood Restaurant        Farmers Market   

  8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
0               Theater    Mexican Restaurant                          Spa  
1                   Bar         Movie Theater                     Beer Bar  
2             BBQ Joint    Italian Restaurant                         Café  
3  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant  
4            Restaurant    Italian Restaurant                       Bakery

In [138]:
toronto_merged.drop('index',axis=1,inplace=True)
toronto_merged.head()

PostalCode            Borough                Neighbourhood   Latitude  \
0        M5A   Downtown Toronto   Harbourfront,  Regent Park  43.654260   
1        M5B   Downtown Toronto    Ryerson,  Garden District  43.657162   
2        M5C   Downtown Toronto               St. James Town  43.651494   
3        M4E       East Toronto                  The Beaches  43.676357   
4        M5E   Downtown Toronto                  Berczy Park  43.644771   

   Longitude  Cluster_Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.360636               0           Coffee Shop                Bakery   
1 -79.378937               0                  Café           Coffee Shop   
2 -79.375418               0           Coffee Shop            Restaurant   
3 -79.293031               0           Coffee Shop     Health Food Store   
4 -79.373306               0           Coffee Shop          Cocktail Bar   

  3rd Most Common Venue 4th Most Common Venue      5th Most Common Venue  \
0                  Park                   Pub                       Café   
1        Clothing Store      Ramen Restaurant  Middle Eastern Restaurant   
2                 Hotel             Gastropub        Japanese Restaurant   
3                   Pub  Other Great Outdoors               Neighborhood   
4                  Café           Cheese Shop                 Steakhouse   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0        Breakfast Spot            Restaurant               Theater   
1               Theater             Gastropub                   Bar   
2        Farmers Market              Beer Bar             BBQ Joint   
3           Yoga Studio         Deli / Bodega  Ethiopian Restaurant   
4    Seafood Restaurant        Farmers Market            Restaurant   

  9th Most Common Venue       10th Most Common Venue  
0    Mexican Restaurant                          Spa  
1         Movie Theater                     Beer Bar  
2    Italian Restaurant                         Café  
3     Electronics Store  Eastern European Restaurant  
4    Italian Restaurant                       Bakery

In [139]:
toronto_merged

PostalCode            Borough  \
0         M5A   Downtown Toronto   
1         M5B   Downtown Toronto   
2         M5C   Downtown Toronto   
3         M4E       East Toronto   
4         M5E   Downtown Toronto   
5         M5G   Downtown Toronto   
6         M6G   Downtown Toronto   
7         M5H   Downtown Toronto   
8         M6H       West Toronto   
9         M5J   Downtown Toronto   
10        M6J       West Toronto   
11        M4K       East Toronto   
12        M5K   Downtown Toronto   
13        M6K       West Toronto   
14        M4L       East Toronto   
15        M5L   Downtown Toronto   
16        M4M       East Toronto   
17        M4N    Central Toronto   
18        M5N    Central Toronto   
19        M4P    Central Toronto   
20        M5P    Central Toronto   
21        M6P       West Toronto   
22        M4R    Central Toronto   
23        M5R    Central Toronto   
24        M6R       West Toronto   
25        M4S    Central Toronto   
26        M5S   Downtown Toronto   
27        M6S       West Toronto   
28        M4T    Central Toronto   
29        M5T   Downtown Toronto   
30        M4V    Central Toronto   
31        M5V   Downtown Toronto   
32        M4W   Downtown Toronto   
33        M5W   Downtown Toronto   
34        M4X   Downtown Toronto   
35        M5X   Downtown Toronto   
36        M4Y   Downtown Toronto   
37        M7Y       East Toronto   

                                        Neighbourhood   Latitude  Longitude  \
0                          Harbourfront,  Regent Park  43.654260 -79.360636   
1                           Ryerson,  Garden District  43.657162 -79.378937   
2                                      St. James Town  43.651494 -79.375418   
3                                         The Beaches  43.676357 -79.293031   
4                                         Berczy Park  43.644771 -79.373306   
5                                  Central Bay Street  43.657952 -79.387383   
6                                            Christie  43.669542 -79.422564   
7                          Adelaide,  King,  Richmond  43.650571 -79.384568   
8                       Dovercourt Village,  Dufferin  43.669005 -79.442259   
9    Harbourfront East,  Toronto Islands,  Union S...  43.640816 -79.381752   
10                          Little Portugal,  Trinity  43.647927 -79.419750   
11                      The Danforth West,  Riverdale  43.679557 -79.352188   
12          Design Exchange,  Toronto Dominion Centre  43.647177 -79.381576   
13     Brockton,  Exhibition Place,  Parkdale Village  43.636847 -79.428191   
14                    The Beaches West,  India Bazaar  43.668999 -79.315572   
15                    Commerce Court,  Victoria Hotel  43.648198 -79.379817   
16                                    Studio District  43.659526 -79.340923   
17                                      Lawrence Park  43.728020 -79.388790   
18                                           Roselawn  43.711695 -79.416936   
19                                   Davisville North  43.712751 -79.390197   
20               Forest Hill North,  Forest Hill West  43.696948 -79.411307   
21                     High Park,  The Junction South  43.661608 -79.464763   
22                                 North Toronto West  43.715383 -79.405678   
23              The Annex,  North Midtown,  Yorkville  43.672710 -79.405678   
24                            Parkdale,  Roncesvalles  43.648960 -79.456325   
25                                         Davisville  43.704324 -79.388790   
26                    Harbord,  University of Toronto  43.662696 -79.400049   
27                                Runnymede,  Swansea  43.651571 -79.484450   
28                       Moore Park,  Summerhill East  43.689574 -79.383160   
29        Chinatown,  Grange Park,  Kensington Market  43.653206 -79.400049   
30   Deer Park,  Forest Hill SE,  Rathnelly,  Sout...  43.686412 -79.400049   
31   CN Tower,  Bathurst Quay,  Island airport,  H...  43.628947 -79.3

In [143]:

# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining clusters

In [145]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
0    Downtown Toronto               0           Coffee Shop   
1    Downtown Toronto               0                  Café   
2    Downtown Toronto               0           Coffee Shop   
3        East Toronto               0           Coffee Shop   
4    Downtown Toronto               0           Coffee Shop   
5    Downtown Toronto               0           Coffee Shop   
6    Downtown Toronto               0         Grocery Store   
7    Downtown Toronto               0                  Café   
8        West Toronto               0              Pharmacy   
9    Downtown Toronto               0              Aquarium   
10       West Toronto               0                   Bar   
11       East Toronto               0      Greek Restaurant   
12   Downtown Toronto               0           Coffee Shop   
13       West Toronto               0           Coffee Shop   
14       East Toronto               0                  Park   
15   Downtown Toronto               0                  Café   
16       East Toronto               0                  Café   
19    Central Toronto               0                   Gym   
21       West Toronto               0         Grocery Store   
22    Central Toronto               0        Clothing Store   
23    Central Toronto               0           Coffee Shop   
24       West Toronto               0        Breakfast Spot   
25    Central Toronto               0        Sandwich Place   
26   Downtown Toronto               0                  Café   
27       West Toronto               0           Pizza Place   
29   Downtown Toronto               0                  Café   
30    Central Toronto               0           Coffee Shop   
31   Downtown Toronto               0        Airport Lounge   
33   Downtown Toronto               0                  Café   
34   Downtown Toronto               0           Coffee Shop   
35   Downtown Toronto               0                  Café   
36   Downtown Toronto               0           Coffee Shop   
37       East Toronto               0    Light Rail Station   

   2nd Most Common Venue 3rd Most Common Venue          4th Most Common Venue  \
0                 Bakery                  Park                            Pub   
1            Coffee Shop        Clothing Store               Ramen Restaurant   
2             Restaurant                 Hotel                      Gastropub   
3      Health Food Store                   Pub           Other Great Outdoors   
4           Cocktail Bar                  Café                    Cheese Shop   
5        Bubble Tea Shop                  Café                Thai Restaurant   
6                   Café                  Park                     Baby Store   
7             Steakhouse           Pizza Place                          Hotel   
8         Discount Store           Supermarket                         Bakery   
9            Coffee Shop                 Hotel                            Bar   
10           Coffee Shop      Asian Restaurant                     Restaurant   
11           Coffee Shop        Ice Cream Shop                      Bookstore   
12                  Café         Deli / Bodega                     Restaurant   
13                  Café        Breakfast Spot                    Yoga Studio   
14        Sandwich Place          Liquor Store             Italian Restaurant   
15           Coffee Shop                 Hotel                      Gastropub   
16           Coffee Shop             Gastropub                         Bakery   
19           Pizza Place         Grocery Store                           Park   
21    Mexican Restaurant                  Café                Thai Restaurant   
22   Sporting Goods Shop           Coffee Shop           Fast Food Restaurant   
23                  Café        Sandwich Place                    Pizza Place   
24             Gift Shop          Dessert Shop                     Restaurant   
25      

In [146]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
28   Central Toronto               1            Playground   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
28  Fast Food Restaurant    Falafel Restaurant           Event Space   

   5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
28  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
28   Dumpling Restaurant               Dog Run         Discount Store

In [147]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
20    Central Toronto               2                  Park   
32   Downtown Toronto               2                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
20                 Trail      Sushi Restaurant         Jewelry Store   
32            Playground                 Trail         Deli / Bodega   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
20           Yoga Studio      Department Store           Event Space   
32    Falafel Restaurant           Event Space  Ethiopian Restaurant   

   8th Most Common Venue        9th Most Common Venue  \
20  Ethiopian Restaurant            Electronics Store   
32     Electronics Store  Eastern European Restaurant   

         10th Most Common Venue  
20  Eastern European Restaurant  
32          Dumpling Restaurant

In [148]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


Borough  Cluster_Labels 1st Most Common Venue  \
18   Central Toronto               3                  Pool   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
18                Garden           Yoga Studio         Deli / Bodega   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
18           Event Space  Ethiopian Restaurant     Electronics Store   

          8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
18  Eastern European Restaurant   Dumpling Restaurant                Dog Run

In [149]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
17   Central Toronto               4              Bus Line   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
17                  Park           Swim School         Deli / Bodega   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
17           Event Space  Ethiopian Restaurant     Electronics Store   

          8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
17  Eastern European Restaurant   Dumpling Restaurant                Dog Run